In [1]:
import pyspark
import pyspark.sql.types as T
import pyspark.sql.functions as F
import requests

sc = pyspark.SparkContext(appName="myAppName")
spark = pyspark.sql.SQLContext(sc)

In [5]:
dataDF_raw = spark.read.json("data.json")

In [3]:
dataDF_raw.count()

1076483

In [6]:
dataDF = dataDF_raw.select(dataDF_raw.repo_name, dataDF_raw.body,\
                           dataDF_raw.created_at.cast('timestamp'), dataDF_raw.closed_at.cast('timestamp'))

In [5]:
dataDF_exa = spark.read.json("C:\\Users\\79241\\Desktop\\MIE1512\\Project\\Data\\akka_akka\\akka_akka_issues_1.json")
data_rows = dataDF_exa.collect()

In [7]:
print(data_rows[0].body)

https://github.com/akka/akka/pull/26633 added actorRefWithAck with 1-1 acks and no buffering (single element buffer).

We could also have a variant with a buffer and OverflowStrategy similar to ActorRefSource and SourceQueue.

Then we could also support batch sending into the stream by having having a function that evaluates if an ack should be sent back for a given element and thereby 1-1 acks are not needed. Could be `ackMessage: Any => Option[Any]`, where the first Any is the incoming message and the second is optionally the ack message.

Then one can define a `Source.actorRefWithAck` with buffer size 100 and use it like:

```
ackMessage = n => if (n % 50 == 0) Some(n) else None

ref ! 1
ref ! 2
ref ! 3
...
ref ! 99
// wait for ack 50 if not already received
ref ! 100
ref ! 101
...
ref ! 149
// wait for ack 100 if not already received
...
```

Inspiration for jmh benchmark can be found in https://github.com/akka/akka/blob/master/akka-bench-jmh/src/main/scala/akka/remote/artery/SendQ

In [3]:
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer

In [7]:
df_clean_1 = dataDF.select('repo_name', 'created_at', 'closed_at', \
                           (lower(regexp_replace('body', "```(.*?)```", "")).alias('body')))

In [8]:
df_clean_1 = df_clean_1.select('repo_name', 'created_at', 'closed_at', \
                           (regexp_replace('body', "@(.*?)", "").alias('body')))

In [9]:
df_clean_1 = df_clean_1.select('repo_name', 'created_at', 'closed_at', \
                           (regexp_replace('body', "#(.*?)", "").alias('body')))

In [10]:
df_clean_1 = df_clean_1.select('repo_name', 'created_at', 'closed_at', \
                           (regexp_replace('body', "www(.*?)", "").alias('body')))

In [11]:
df_clean_1 = df_clean_1.select('repo_name', 'created_at', 'closed_at', \
                           (regexp_replace('body', "http(.*?)", "").alias('body')))

In [12]:
data_DF_final = df_clean_1.na.drop(subset=["body"])

In [14]:
data_DF_final.write.csv(path='\clean_data.csv', mode="append", header="true")

Py4JJavaError: An error occurred while calling o130.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperation

In [25]:
sc.version

'2.4.0'

In [15]:
df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('clean_tweet.csv')